In [6]:
from __future__ import print_function
# key libs
import numpy as np
import re
import nltk
import pandas as pd
import glob
import codecs

# nlp libs
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 

# processing
from sklearn.model_selection import train_test_split

# LDA
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.decomposition import LatentDirichletAllocation
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF


# bring in my pickled vectorizers
import pickle
import dill
# import import_ipynb
# from process_tokenize import my_tokenizer

In [7]:
# what are we interestd in
# set this up to read the correct set of files we are interested in

n_gram = '1gm' # options - 1gm , 2gm
stem_type = 'lemma' # options - lemma,snow

In [8]:
# set random state
RSEED = 0

### 1. OPEN - TERM FREQUENCY FILES 
i.e. output of CountVectorizer

In [9]:
tf_train = pd.read_csv('../data/vectors/cv_'+ n_gram +'_'+ stem_type +'_train.csv', compression = 'gzip')
tf_train = tf_train.drop(columns='Unnamed: 0')

tf_test = pd.read_csv('../data/vectors/cv_'+ n_gram +'_'+ stem_type +'_test.csv', compression = 'gzip')
tf_test = tf_test.drop(columns='Unnamed: 0')

In [10]:
tf_train.head()

,book_title,author_name,book_location,aaron,aback,abandon,abandoned,abandoning,abandonment,abasement,...,zealand,zealous,zealously,zenith,zephyr,zero,zest,zigzag,zone,zoological
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0,0,2,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,3,0,0


In [11]:
# load the appropriate tf_vectorizer
tf_vectorizer = dill.load(open('../data/vectors/tf_vectorizer_'+ stem_type + '_' + str(n_gram), 'rb'))

### 2. OPEN - TERM FREQUENCY INVERSE DOCUMENT FREQUENCY FILES
i.e output of TfidfVectorizer 

In [64]:
tfid_train = pd.read_csv('../data/vectors/tfid_'+ n_gram +'_'+ stem_type +'_train.csv',compression = 'gzip')
tfid_train = tfid_train.drop(columns='Unnamed: 0')

tfid_test = pd.read_csv('../data/vectors/tfid_'+ n_gram +'_'+ stem_type +'_test.csv', compression = 'gzip')
tfid_test = tfid_test.drop(columns='Unnamed: 0')

In [65]:
tfid_train.head()

,book_title,author_name,book_location,aaron,aback,abandon,abandoned,abandoning,abandonment,abasement,...,zealand,zealous,zealously,zenith,zephyr,zero,zest,zigzag,zone,zoological
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.000000
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.0,0.0,0.004126,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.0,0.005299
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.0,0.0,0.006395,0.002774,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00448,0.00000,0.0,0.000000
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.005872,...,0.0,0.0,0.0,0.0,0.0,0.00554,0.00000,0.00000,0.0,0.000000
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.0,0.0,0.000000,0.013355,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.00000,0.00000,0.03942,0.0,0.000000


In [66]:
# load the appropriate tf_vectorizer
#tfid_vectorizer = pickle.load(open('../data/vectors/tfid_vectorizer_'+ stem_type +'_'+ n_gram +'.pkl', 'rb'))
tfid_vectorizer = dill.load(open('../data/vectors/tfid_vectorizer_'+ stem_type + '_' + str(n_gram), 'rb'))

### 3. HELPER FUNCTIONS

In [17]:
#function to display topics
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print (f'Topic: {topic_idx}')
        print (" , ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [56]:
#function to display books for a topic
def find_topic_books(df,number):
    topics = list(df.columns)[3:]
    #print(topics)
    for t in topics:
        temp = df.sort_values(by=t,ascending=False)
        print('\n------'+t+'-------') 
        print(temp.iloc[0:number+1,0:2])    

### 4. DIMENSIONALITY REDUCTION - TOPIC MODELLING
1. LDA
2. NMF
3. LSA / LSI ? - These are rather old and don't give good results


use the Term Frequency matrix as Term Frequency Inverse Document Frequency does not work well

In [19]:
# set how many topics you want
topic_number = 20

### A. LDA with TF
LatentDirichletAllocation with the Term Frequency Allocation
Use the Term Frequency matrix as Term Frequency Inverse Document Frequency does not work well
LDA can only use raw term counts for LDA because it is a probabilistic graphical model

In [20]:
# Run LDA 
lda_tf = LatentDirichletAllocation(learning_method= 'batch',n_components=topic_number, random_state=RSEED, batch_size=128,n_jobs=-1)
lda_tf.fit(tf_train.iloc[:,3:])

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=20, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [21]:
# transform vectors
vec_lda_train = lda_tf.transform(tf_train.iloc[:,3:])
vec_lda_test = lda_tf.transform(tf_test.iloc[:,3:])

In [22]:
dill.dump(lda_tf, open('../data/vectors/lda_'+ stem_type + '_' + n_gram,'wb'))

In [23]:
print('------PERPLEXITY------')
print(lda_tf.perplexity(tf_train.iloc[:,3:]))
print(lda_tf.perplexity(tf_test.iloc[:,3:]))

------PERPLEXITY------
4945.746276061271
5220.032446671448


In [49]:
# get the topics and words for the topics in lda
no_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names()
display_topics(lda_tf, tf_feature_names, no_top_words)

Topic: 0
thou , thy , thee , king , sir , lord , ti , hath , lady , god
Topic: 1
sir , lord , lady , manner , person , father , letter , family , master , gentleman
Topic: 2
girl , don , answered , sat , horse , suddenly , replied , rose , strange , slowly
Topic: 3
sea , boat , tree , ship , wind , island , big , dog , sun , red
Topic: 4
poet , poem , art , nature , line , footnote , beauty , note , letter , poetry
Topic: 5
king , father , sir , prince , horse , english , master , town , england , lord
Topic: 6
sir , lady , gentleman , dear , cried , answered , madame , miss , table , window
Topic: 7
tom , dick , boy , don , sam , answered , cried , jim , yes , didn
Topic: 8
don , mr , miss , yes , sir , boy , didn , girl , ain , won
Topic: 9
girl , mother , boy , peter , don , cried , anne , story , tree , dear
Topic: 10
money , class , law , trade , business , price , land , american , value , public
Topic: 11
government , general , power , law , war , public , lord , act , person , 

In [30]:
# add back details
lda_train = pd.DataFrame(vec_lda_train, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
lda_train.insert(loc=0, column='book_location', value=tf_train['book_location'])
lda_train.insert(loc=0, column='author_name', value=tf_train['author_name'])
lda_train.insert(loc=0, column='book_title', value=tf_train['book_title'])
lda_train.to_csv('../data/vectors/lda_'+ n_gram+'_'+ stem_type +'_train.csv')
lda_train.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.000006,0.000006,0.000006,0.056614,0.042713,0.000006,0.164503,...,0.066676,0.000006,0.000006,0.000006,0.000006,0.074134,0.000006,0.079391,0.000006,0.179807
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.000003,0.021875,0.000003,0.000003,0.000003,0.000003,0.034393,...,0.004022,0.000003,0.000003,0.000003,0.004022,0.060004,0.172095,0.000003,0.000003,0.112089
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.000004,0.009116,0.076108,0.098339,0.008048,0.055905,0.199755,...,0.015974,0.008912,0.010840,0.040935,0.000004,0.105113,0.047032,0.000004,0.000004,0.131171
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.000009,0.000009,0.412892,0.166950,0.017638,0.000009,0.024758,...,0.000009,0.000145,0.051583,0.000009,0.000009,0.000009,0.000009,0.000009,0.000009,0.012090
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.002563,0.082916,0.020869,0.071870,0.000009,0.000009,0.440103,...,0.000159,0.013008,0.017302,0.005694,0.070540,0.033966,0.009803,0.012162,0.007731,0.107179


In [31]:
# add the book details and write to file for test
lda_test = pd.DataFrame(vec_lda_test, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
lda_test.insert(loc=0, column='book_location', value=tf_test['book_location'])
lda_test.insert(loc=0, column='author_name', value=tf_test['author_name'])
lda_test.insert(loc=0, column='book_title', value=tf_test['book_title'])
lda_test.to_csv('../data/vectors/lda_'+ n_gram+'_'+ stem_type +'_test.csv')
lda_test.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,Great Catherine,George Bernard Shaw,../data/gutenberg/George Bernard Shaw___Great ...,0.088418,0.000012,0.000012,0.000012,0.003382,0.000692,0.316283,...,0.000012,0.000012,0.000012,0.038180,0.000012,0.170649,0.129748,0.067013,0.000012,0.000012
1,Short Cruises,William Wymark Jacobs,../data/gutenberg/William Wymark Jacobs___Shor...,0.000003,0.000003,0.000003,0.013034,0.000003,0.000003,0.001170,...,0.000003,0.000003,0.006404,0.000572,0.000003,0.000003,0.048132,0.000003,0.000003,0.000003
2,Richard Dare's Venture,Edward Stratemeyer,../data/gutenberg/Edward Stratemeyer___Richard...,0.000004,0.000004,0.000004,0.000004,0.000004,0.156210,0.016494,...,0.046273,0.000004,0.000004,0.000004,0.000004,0.000004,0.070800,0.000004,0.000004,0.000004
3,William Harvey And The Discovery Of The Circul...,Thomas Henry Huxley,../data/gutenberg/Thomas Henry Huxley___Willia...,0.000025,0.000025,0.000025,0.000025,0.000025,0.028644,0.000025,...,0.006689,0.048928,0.000025,0.000025,0.539049,0.350062,0.000025,0.000025,0.008489,0.000025
4,Beyond,John Galsworthy,../data/gutenberg/John Galsworthy___Beyond.txt,0.004891,0.000002,0.097843,0.056868,0.000896,0.000002,0.071941,...,0.000002,0.000002,0.000002,0.000002,0.000002,0.000002,0.007421,0.000002,0.002691,0.453285


In [58]:
# find documents for topic
no_documents = 10
find_topic_books(lda_train,no_documents)


------topic_1-------
                       book_title                 author_name
180               Comic Tragedies           Louisa May Alcott
2140          The Last Tournament               Lord Tennyson
1321                Helen of Troy                 Andrew Lang
1594                Count Alarcos           Benjamin Disraeli
1707           Idylls of the King               Lord Tennyson
1915                      Marmion            Sir Walter Scott
1045                        Lamia                  John Keats
1203                Chamber Music                 James Joyce
524   Sonnets from the Portuguese  Elizabeth Barrett Browning
320                        Mosada        William Butler Yeats
668                     Cleopatra         Henry Rider Haggard

------topic_2-------
                    book_title       author_name
120      Trial of Duncan Terig  Sir Walter Scott
700   The Heart of Mid-Lothian  Sir Walter Scott
1311                   Rob Roy  Sir Walter Scott
776        Bride

In [20]:
# setup for visualization
vec_tf=csr_matrix(tf_train.iloc[:,3:].values)
pyLDAvis.sklearn.prepare(lda_tf,vec_tf,tf_vectorizer)

PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
16     8.839396        1       1  0.014473  0.002055
19     7.963302        1       2  0.027382  0.011075
6      7.370487        1       3  0.034977 -0.026367
3      5.863017        1       4  0.143264  0.015960
2      5.420233        1       5  0.103922  0.035415
8      5.410390        1       6  0.124268  0.045761
0      5.144984        1       7  0.081296 -0.193153
1      5.143561        1       8 -0.072498 -0.014674
4      4.931317        1       9 -0.089414 -0.116005
13     4.882063        1      10  0.034352  0.125792
12     4.881788        1      11  0.056797  0.035286
15     4.742639        1      12 -0.151256 -0.002437
14     4.504723        1      13 -0.177772  0.098919
9      4.292434        1      14  0.170285 -0.033988
5      4.180314        1      15  0.038860 -0.013553
11     4.162316        1      16 -0.206627  0.014305
17     3.844086        1      17 -0.077391 -0.014938
18     3.085535        1      18 -0.052245 -0.173385
7      2.685189        1      19  0.168445  0.112041
10     2.652227        1      20 -0.171117  0.091892, topic_info=      Category           Freq         Term          Total  loglift  logprob
term                                                                       
7167   Default   82107.000000          god   82107.000000  30.0000  30.0000
4937   Default  151248.000000          don  151248.000000  29.0000  29.0000
10562  Default  119970.000000           mr  119970.000000  28.0000  28.0000
16409  Default   45703.000000         thou   45703.000000  27.0000  27.0000
9218   Default   75647.000000         king   75647.000000  26.0000  26.0000
1882   Default   84022.000000          boy   84022.000000  25.0000  25.0000
16482  Default   35142.000000          thy   35142.000000  24.0000  24.0000
9296   Default  118565.000000         lady  118565.000000  23.0000  23.0000
16585  Default   26892.000000          tom   26892.000000  22.0000  22.0000
14768  Default  119344.000000          sir  119344.000000  21.0000  21.0000
4484   Default   20520.000000         dick   20520.000000  20.0000  20.0000
16341  Default   29479.000000         thee   29479.000000  19.0000  19.0000
9732   Default   80575.000000         lord   80575.000000  18.0000  18.0000
10351  Default   77914.000000         miss   77914.000000  17.0000  17.0000
7076   Default   79964.000000         girl   79964.000000  16.0000  16.0000
2301   Default   57550.000000      captain   57550.000000  15.0000  15.0000
4496   Default   37492.000000         didn   37492.000000  14.0000  14.0000
15191  Default   19043.000000       specie   19043.000000  13.0000  13.0000
14553  Default   40633.000000         ship   40633.000000  12.0000  12.0000
6991   Default   59828.000000      general   59828.000000  11.0000  11.0000
3750   Default   51106.000000        cried   51106.000000  10.0000  10.0000
18397  Default   77595.000000          yes   77595.000000   9.0000   9.0000
656    Default   58777.000000     answered   58777.000000   8.0000   8.0000
6168   Default  112137.000000       father  112137.000000   7.0000   7.0000
1763   Default   38726.000000         boat   38726.000000   6.0000   6.0000
8437   Default   24909.000000       indian   24909.000000   5.0000   5.0000
7211   Default   24062.000000   government   24062.000000   4.0000   4.0000
9398   Default   47809.000000          law   47809.000000   3.0000   3.0000
10439  Default   54381.000000        money   54381.000000   2.0000   2.0000
15124  Default   38448.000000         soul   38448.000000   1.0000   1.0000
...        ...            ...          ...            ...      ...      ...
8718   Topic20    2006.450963  institution    5502.443929   2.6209  -6.4460
17232  Topic20    3064.505157       united   10028.287712   2.4442  -6.0225
8400   Topic20    2399.134823     increase    7219.692457   2.5281  -6.2672
3590   Topic20    2974.989305         cost   10964.7179

### B. NMF WITH TFID
NMF workd better with TFID

In [67]:
# Run NMF
nmf_tfid = NMF(n_components=topic_number, random_state=RSEED, max_iter=500)
nmf_tfid.fit(tfid_train.iloc[:,3:])

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=500,
  n_components=20, random_state=0, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [68]:
# transform vectors
vec_nmf_train = nmf_tfid.transform(tfid_train.iloc[:,3:])
vec_nmf_test = nmf_tfid.transform(tfid_test.iloc[:,3:])

In [69]:
dill.dump(nmf_tfid, open('../data/vectors/nmf_'+ stem_type + '_' + n_gram,'wb'))

In [70]:
no_top_words = 10
tfid_feature_names = tfid_vectorizer.get_feature_names()
display_topics(nmf_tfid, tfid_feature_names, no_top_words)

Topic: 0
answered , cried , madame , replied , monsieur , lip , girl , exclaimed , smile , continued
Topic: 1
thou , thy , thee , hath , ti , thine , hast , heaven , soul , art
Topic: 2
art , poet , public , character , literature , english , government , modern , society , author
Topic: 3
mr , husband , didn , couldn , wouldn , isn , won , doesn , wasn , girl
Topic: 4
sir , lord , gentleman , john , george , duke , harry , master , london , james
Topic: 5
s , wot , arter , ave , ginger , sam , bob , agin , ead , skipper
Topic: 6
captain , ship , boat , deck , island , vessel , board , crew , sail , mate
Topic: 7
dick , tom , sam , rover , baxter , dan , fred , cried , answered , cadet
Topic: 8
green , bird , hill , flower , sky , wood , mountain , sweet , blue , dream
Topic: 9
god , christ , lord , sin , jesus , soul , heaven , christian , holy , church
Topic: 10
specie , animal , darwin , plant , hypothesis , science , organic , natural , modification , bird
Topic: 11
officer , army 

In [71]:
# add back details
nmf_train = pd.DataFrame(vec_nmf_train, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
nmf_train.insert(loc=0, column='book_location', value=tfid_train['book_location'])
nmf_train.insert(loc=0, column='author_name', value=tfid_train['author_name'])
nmf_train.insert(loc=0, column='book_title', value=tfid_train['book_title'])
nmf_train.to_csv('../data/vectors/nmf_'+ n_gram+'_'+ stem_type +'_train.csv')
nmf_train.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,Behind the Beyond,Stephen Leacock,../data/gutenberg/Stephen Leacock___Behind the...,0.024754,0.0,0.030954,0.032591,0.081775,0.000000,0.000000,...,0.000000,0.000000,0.072496,0.000000,0.043274,0.000000,0.005502,0.000000,0.053511,0.000891
1,Tommy and Co,Jerome Klapka Jerome,../data/gutenberg/Jerome Klapka Jerome___Tommy...,0.010176,0.0,0.015237,0.047607,0.006928,0.001250,0.000000,...,0.000000,0.000000,0.003213,0.000000,0.000000,0.000000,0.241563,0.077373,0.007175,0.004169
2,Winsome Winnie and other New Nonsense Novels,Stephen Leacock,../data/gutenberg/Stephen Leacock___Winsome Wi...,0.044107,0.0,0.014085,0.000000,0.083100,0.000000,0.013311,...,0.003250,0.007783,0.030975,0.003627,0.005625,0.012087,0.013309,0.053859,0.000000,0.019906
3,The Moccasin Ranch,Hamlin Garland,../data/gutenberg/Hamlin Garland___The Moccasi...,0.004341,0.0,0.000000,0.022038,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.043628,0.000000,0.000330,0.040634,0.000000,0.000000,0.000000,0.000000
4,Three Ghost Stories,Charles Dickens,../data/gutenberg/Charles Dickens___Three Ghos...,0.063080,0.0,0.018698,0.000363,0.090716,0.001838,0.010200,...,0.025048,0.003055,0.004595,0.000000,0.004226,0.008518,0.000547,0.056877,0.000000,0.006891


In [72]:
# add back details
nmf_test = pd.DataFrame(vec_nmf_test, columns=['topic_'+ str(i)for i in range(1,topic_number+1)])
nmf_test.insert(loc=0, column='book_location', value=tfid_test['book_location'])
nmf_test.insert(loc=0, column='author_name', value=tfid_test['author_name'])
nmf_test.insert(loc=0, column='book_title', value=tfid_test['book_title'])
nmf_test.to_csv('../data/vectors/nmf_'+ n_gram+'_'+ stem_type +'_test.csv')
nmf_test.head()

,book_title,author_name,book_location,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,...,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20
0,Great Catherine,George Bernard Shaw,../data/gutenberg/George Bernard Shaw___Great ...,0.013377,0.001834,0.010979,0.002782,0.006235,0.000320,0.011939,...,0.000000,0.009849,0.001425,0.005960,0.000819,0.0,0.003740,0.011713,0.060561,0.000653
1,Short Cruises,William Wymark Jacobs,../data/gutenberg/William Wymark Jacobs___Shor...,0.011489,0.000000,0.000000,0.053352,0.003168,0.394411,0.006154,...,0.000000,0.000000,0.008906,0.000000,0.013697,0.0,0.004149,0.037200,0.000000,0.000000
2,Richard Dare's Venture,Edward Stratemeyer,../data/gutenberg/Edward Stratemeyer___Richard...,0.008674,0.000000,0.000000,0.018326,0.047545,0.000000,0.000000,...,0.000000,0.000000,0.034548,0.039140,0.000000,0.0,0.000000,0.000000,0.000000,0.000247
3,William Harvey And The Discovery Of The Circul...,Thomas Henry Huxley,../data/gutenberg/Thomas Henry Huxley___Willia...,0.003790,0.000260,0.023789,0.000000,0.000000,0.000000,0.002772,...,0.063213,0.005000,0.000000,0.006096,0.000000,0.0,0.000000,0.000000,0.000000,0.002766
4,Beyond,John Galsworthy,../data/gutenberg/John Galsworthy___Beyond.txt,0.095430,0.000000,0.000000,0.034680,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.049041,0.000000,0.001582,0.0,0.004010,0.057867,0.000000,0.000000


In [73]:
# find documents for topic
no_documents = 10
find_topic_books(nmf_train,no_documents)


------topic_1-------
                       book_title                author_name
254                 The Missioner  Edward Phillips Oppenheim
550              The Great Secret  Edward Phillips Oppenheim
1037                  The Avenger  Edward Phillips Oppenheim
1854            The Yellow Crayon  Edward Phillips Oppenheim
2127       The Vanished Messenger  Edward Phillips Oppenheim
2179         Mysterious Mr. Sabin  Edward Phillips Oppenheim
984             The Master Mummer  Edward Phillips Oppenheim
564                      Berenice  Edward Phillips Oppenheim
1319                        Havoc  Edward Phillips Oppenheim
1672  The Trampling of the Lilies            Rafael Sabatini
791             Mr. Marx's Secret  Edward Phillips Oppenheim

------topic_2-------
                             book_title                 author_name
180                     Comic Tragedies           Louisa May Alcott
2158  The Poetical Works of John Milton                 John Milton
524         Sonnets 

In [74]:
# setup for visualization
vec_tfid=csr_matrix(tfid_train.iloc[:,3:].values)
pyLDAvis.sklearn.prepare(nmf_tfid,vec_tfid,tfid_vectorizer)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      12.747954        1       1 -0.217977 -0.082265
0       8.084530        1       2 -0.027522 -0.088157
4       7.843532        1       3 -0.085834 -0.043873
8       6.874480        1       4  0.015135  0.131293
15      6.614661        1       5 -0.020437 -0.001372
12      6.190448        1       6  0.138922 -0.070451
17      5.724203        1       7  0.029689 -0.092775
11      5.613333        1       8 -0.137938 -0.039769
3       5.518599        1       9  0.067304 -0.186479
13      5.153137        1      10 -0.090067  0.167443
9       4.972857        1      11 -0.160590  0.153528
10      4.581926        1      12 -0.167089 -0.099130
1       4.543971        1      13 -0.092724  0.235424
6       4.499612        1      14 -0.005772 -0.017890
18      3.134909        1      15 -0.125556 -0.037543
7       2.131263        1      16  0.179619 -0.012514
16      1.836390        1      17  0.242580  0.065428
14      1.798166        1      18  0.176993  0.156110
19      1.392345        1      19  0.010779 -0.113427
5       0.743683        1      20  0.270485 -0.023581, topic_info=      Category        Freq          Term       Total  loglift  logprob
term                                                                  
10481  Default  217.000000            mr  217.000000  30.0000  30.0000
4455   Default   93.000000          dick   93.000000  29.0000  29.0000
13849  Default   69.000000             s   69.000000  28.0000  28.0000
7110   Default  123.000000           god  123.000000  27.0000  27.0000
8090   Default   73.000000  illustration   73.000000  26.0000  26.0000
9156   Default  119.000000          king  119.000000  25.0000  25.0000
11694  Default   72.000000         peter   72.000000  24.0000  24.0000
16456  Default   75.000000           tom   75.000000  23.0000  23.0000
16281  Default   82.000000          thou   82.000000  22.0000  22.0000
10273  Default   96.000000          miss   96.000000  21.0000  21.0000
13915  Default   60.000000           sam   60.000000  20.0000  20.0000
16353  Default   71.000000           thy   71.000000  19.0000  19.0000
7822   Default   39.000000        holmes   39.000000  18.0000  18.0000
16214  Default   57.000000          thee   57.000000  17.0000  17.0000
9002   Default   38.000000         jimmy   38.000000  16.0000  16.0000
14656  Default  100.000000           sir  100.000000  15.0000  15.0000
9658   Default   89.000000          lord   89.000000  14.0000  14.0000
2283   Default   69.000000       captain   69.000000  13.0000  13.0000
13766  Default   34.000000         rover   34.000000  12.0000  12.0000
12316  Default   55.000000        prince   55.000000  11.0000  11.0000
18131  Default   25.000000           wot   25.000000  10.0000  10.0000
2628   Default   38.000000        christ   38.000000   9.0000   9.0000
4467   Default   59.000000          didn   59.000000   8.0000   8.0000
14443  Default   48.000000          ship   48.000000   7.0000   7.0000
7021   Default   77.000000          girl   77.000000   6.0000   6.0000
1094   Default   40.000000          aunt   40.000000   5.0000   5.0000
1596   Default   24.000000         billy   24.000000   4.0000   4.0000
6978   Default   41.000000        george   41.000000   3.0000   3.0000
1133   Default   19.000000           ave   19.000000   2.0000   2.0000
1752   Default   46.000000          boat   46.000000   1.0000   1.0000
...        ...         ...           ...         ...      ...      ...
820    Topic20    2.211350           ard    4.212092   4.2570  -5.3812
3255   Topic20    1.370778      conjurer    2.815935   4.1814  -5.8594
12038  Topic20    2.740771          pore    5.635389   4.1805  -5.1666
11120  Topic20    2.040539          ouse    4.217434   4.1753  -5.4616
8991   Topic20    0.919531         jetty    1.972628   4.1381  -6.2587
13834  Topic20    1.244858        russet    2.701664   4.1265  -5.9558
